In [ ]:
! pip install git+http://github.com/firekind/athena &> /dev/null

In [3]:
import os
os.chdir("/content/drive/My Drive/sign-language-model")

In [4]:
from dataset import ASLSignLanguage
from athena.datasets import train_val_split
from athena import Experiments, ClassificationSolver
from athena.models import ResNet18
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
import torch.nn.functional as F

In [5]:
dataset = ASLSignLanguage("./data/asl_alphabet", use_default_transforms=True)
train_loader, val_loader = train_val_split(dataset, 64, val_split=0.2, use_default_val_transform=True)

In [6]:
exps = (
    Experiments.builder()
    .name("Sign Language")
    .log_directory("./logs")
    .add("ResNet-18")
        .data()
            .train_loader(train_loader)
            .val_loader(val_loader)
        .solver(ClassificationSolver)
            .epochs(50)
            .model(ResNet18(num_classes=29))
            .optimizer(optim.SGD, lr=1e-4, momentum=0.9)
        .build()
    .build()
)